<a href="https://colab.research.google.com/github/neSTORY/Deeplearning_Zero_to_All/blob/master/CNN%EC%9C%BC%EB%A1%9C_mnist_%EA%B0%96%EA%B3%A0_%EB%86%80%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [36]:
# Set Hyper parameters
lr = 1e-3
epochs = 15
batch_size = 100

current_dir = os.getcwd()
ckpt_dir_name = "checkpoints"
model_dir_name = "mnist_cnn_seq"

checkpoint_dir = os.path.join(current_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

In [37]:
checkpoint_dir

'/content/checkpoints/mnist_cnn_seq'

In [38]:
checkpoint_prefix

'/content/checkpoints/mnist_cnn_seq/mnist_cnn_seq'

In [39]:
# load dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

# preprocessing data
train_images = train_images.astype(np.float32)/255.0
test_images = test_images.astype(np.float32)/255.0
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).\
                                                  shuffle(buffer_size=100000).\
                                                  batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).\
                                                 batch(batch_size)

In [40]:
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, Dropout
relu = keras.activations.relu

In [41]:
def create_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=3, activation=relu, padding="same", input_shape=(28,28,1)))
  model.add(MaxPooling2D(padding="same"))
  
  model.add(Conv2D(64, kernel_size=3, activation=relu, padding="same"))
  model.add(MaxPooling2D(padding="same"))

  model.add(Conv2D(128, kernel_size=3, activation=relu, padding="same"))
  model.add(MaxPooling2D(padding="same"))

  model.add(Flatten())
  model.add(Dense(256, activation=relu))
  model.add(Dropout(0.4))
  model.add(Dense(10))

  return model

In [42]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 128)       

In [46]:
def loss_fn(model, images, labels):
  logits = model(images, training=True) # training => dropout 적용
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                labels=labels))
  return loss

def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)

In [51]:
optimizer = keras.optimizers.Adam(learning_rate=lr)

def evaluate(model, images, labels):
  logits = model(images, training=False)
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

checkpoint = tf.train.Checkpoint(cnn=model)

In [53]:
# training
for epoch in range(epochs):
  avg_loss, avg_train_acc, avg_test_acc, train_step, test_step = 0,0,0,0,0
  start_time = time.time()

  for images, labels, in train_dataset:

    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.variables))
    loss = loss_fn(model, images, labels)
    acc = evaluate(model, images, labels)
    avg_loss += loss
    avg_train_acc += acc
    train_step +=1

  avg_loss = avg_loss / train_step
  avg_train_acc = avg_train_acc/train_step

  for images, labels in test_dataset:
    acc = evaluate(model, images, labels)
    avg_test_acc = avg_test_acc + acc
    test_step +=1
  acg_test_acc = avg_test_acc / test_step
  end_time = time.time()
  total_time = end_time-start_time

  print("EPOCH : {:4}, Loss : {:6.4f}, Train Accuracy : {:6.4f}, Test Accuracy : {:6.4f}, time : {:6.2f}".format(epoch, avg_loss, avg_train_acc, avg_test_acc, total_time))

EPOCH :    0, Loss : 0.0070, Train Accuracy : 0.9992, Test Accuracy : 99.3800, time :  19.08
EPOCH :    1, Loss : 0.0062, Train Accuracy : 0.9992, Test Accuracy : 99.2400, time :  18.83
EPOCH :    2, Loss : 0.0053, Train Accuracy : 0.9994, Test Accuracy : 99.2700, time :  19.15
EPOCH :    3, Loss : 0.0053, Train Accuracy : 0.9995, Test Accuracy : 99.3700, time :  18.94
EPOCH :    4, Loss : 0.0045, Train Accuracy : 0.9995, Test Accuracy : 99.2900, time :  18.80
EPOCH :    5, Loss : 0.0036, Train Accuracy : 0.9996, Test Accuracy : 99.2200, time :  18.72
EPOCH :    6, Loss : 0.0035, Train Accuracy : 0.9997, Test Accuracy : 99.3100, time :  18.92
EPOCH :    7, Loss : 0.0041, Train Accuracy : 0.9996, Test Accuracy : 99.2300, time :  19.11
EPOCH :    8, Loss : 0.0035, Train Accuracy : 0.9997, Test Accuracy : 99.3500, time :  18.87
EPOCH :    9, Loss : 0.0024, Train Accuracy : 0.9998, Test Accuracy : 99.3400, time :  18.85
EPOCH :   10, Loss : 0.0030, Train Accuracy : 0.9998, Test Accuracy : 